In [1]:
from collections import defaultdict
from collections import Counter
import csv
from __future__ import print_function
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF
from sklearn.datasets import fetch_20newsgroups
import re
import random
import numpy as np

We need to go through all of our tweets and store them. 

In [2]:
democrats = [];
with open('./dem_tweets.txt', 'r') as f:
    democrats = f.read().splitlines();
    
republicans = [];
with open('./rep_tweets.txt', 'r') as f:
    republicans = f.read().splitlines();
    
random.shuffle(republicans)
republicans = republicans[0:len(democrats)]

Getting relevant hashtags from tweets.

In [3]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=4000,
                                stop_words='english')
def removeHTTPS(l):
    return map(lambda x: re.sub(r"(?:\@|https?\://)\S+", "", x), l)

def removeRT(l):
    return map(lambda x: re.sub(r"(RT )", "", x), l)

democrats = removeHTTPS(democrats)
democrats = removeRT(democrats)

republicans = removeHTTPS(republicans)
republicans = removeRT(republicans)



In [4]:
hashtags_dem_count = {};
hashtags_dem_text = {};
hashtags_rep_count = {};
hashtags_rep_text = {};

for tweet in democrats:
    h = re.findall(r"#(\w+)", tweet);
    h = map(lambda x:x.lower(),h);
    for i in h:
        if hashtags_dem_count.get(i) is None:
            hashtags_dem_count[i] = 1;
            hashtags_dem_text[i] = [tweet];
        else:
            hashtags_dem_count[i] += 1;
            hashtags_dem_text[i].append(tweet);
            
for tweet in republicans:
    h = re.findall(r"#(\w+)", tweet);
    h = map(lambda x:x.lower(),h);
    for i in h:
        if hashtags_rep_count.get(i) is None:
            hashtags_rep_count[i] = 1;
            hashtags_rep_text[i] = [tweet];
        else:
            hashtags_rep_count[i] += 1;
            hashtags_rep_text[i].append(tweet);
            
demHashtagCount = Counter(hashtags_dem_count);
repHashtagCount = Counter(hashtags_rep_count);

print("Democrats:")
for k, v in demHashtagCount.most_common(10):
    print('%s: %i' % (k, v))
    
print("")
print("Republicans:")
for k, v in repHashtagCount.most_common(10):
    print('%s: %i' % (k, v))

Democrats:
p2: 41304
gopdebate: 14396
actonclimate: 12996
tcot: 11933
obamacare: 11886
gop: 11465
sotu: 10223
raisethewage: 9053
getcovered: 8881
demdebate: 8352

Republicans:
tcot: 158915
teaparty: 53737
gop: 31150
tlot: 25147
obamacare: 22124
gophers: 13970
gopdebate: 13509
p2: 13062
ff: 11620
sgp: 11134


In [5]:
num_hashtags = 20;

d = [ seq[0] for seq in demHashtagCount.most_common(500) ]
r = [ seq[0] for seq in repHashtagCount.most_common(500) ]

intersection = list(set(d) & set(r));
popular_hashtags = {};
for i in intersection:
    popular_hashtags[i] = hashtags_dem_count[i] + hashtags_rep_count[i];

pop_h_counter = Counter(popular_hashtags);
popular_hashtags = pop_h_counter.most_common(num_hashtags);
print("Popular Hashtags:")
for k, v in popular_hashtags:
    print('%s: %i' % (k, v))

Popular Hashtags:
tcot: 170848
teaparty: 55641
p2: 54366
gop: 42615
obamacare: 34010
gopdebate: 27905
tlot: 26603
ff: 18710
sotu: 16850
obama: 14579
demdebate: 11257
uniteblue: 10452
jobs: 10425
scotus: 9679
nhpolitics: 9154
mapoli: 8970
aca: 8117
sayfie: 7882
txlege: 7865
utpol: 7715


In [6]:
# calculations for first hashtag
hashtag = popular_hashtags[0][0];
dem_tweets0 = hashtags_dem_text[hashtag];
rep_tweets0 = hashtags_rep_text[hashtag];
num_dem = len(dem_tweets0);
num_rep = len(rep_tweets0);

# compute score of unigram u towards affiliation A
p_dem = len(dem_tweets0)*1.0/(len(rep_tweets0) + len(dem_tweets0));
p_rep = len(rep_tweets0)*1.0/(len(rep_tweets0) + len(dem_tweets0));

count_vec = CountVectorizer(max_df=0.8,  stop_words='english', min_df = 0.0001)

tweets_v = count_vec.fit_transform(dem_tweets0 + rep_tweets0).toarray();

terms = count_vec.get_feature_names();

term_dict = {}
for i in range(len(terms)):
    term_dict[terms[i]] = i
    
index_to_vocab = {i:v for i, v in enumerate(count_vec.get_feature_names())}

In [7]:
dem_tweets_v = tweets_v[0:num_dem,:];
rep_tweets_v = tweets_v[num_dem:,:];
sd = p_dem*np.log(np.divide(np.sum(dem_tweets_v, axis = 0),np.sum(rep_tweets_v, axis = 0)));
sr = p_rep*np.log(np.divide(np.sum(rep_tweets_v, axis = 0), np.sum(dem_tweets_v, axis = 0)));

/Users/abigailshchur/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: RuntimeWarning: divide by zero encountered in divide
  app.launch_new_instance()
/Users/abigailshchur/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()
/Users/abigailshchur/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: RuntimeWarning: divide by zero encountered in divide
/Users/abigailshchur/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: RuntimeWarning: divide by zero encountered in log


In [9]:
import math

u_score = {};
u_score_label = {};
for i in range(0, len(sd)):
    sdvalid = not math.isnan(sd[i]) and not math.isinf(sd[i]) and sd[i] !=0;
    srvalid = not math.isnan(sr[i]) and not math.isinf(sr[i]) and sr[i] !=0;
    if (sdvalid and srvalid):
        if (sd[i] > sr[i]):
            u_score[index_to_vocab[i]] = sd[i];
            u_score_label[index_to_vocab[i]] = "d";
        else:
            u_score[index_to_vocab[i]] = sr[i];
            u_score_label[index_to_vocab[i]] = "r";
    elif (sdvalid):
        u_score[index_to_vocab[i]] = sd[i];
        u_score_label[index_to_vocab[i]] = "d";
    elif (srvalid):
        u_score[index_to_vocab[i]] = sr[i];
        u_score_label[index_to_vocab[i]] = "r";

In [14]:
mpscore_dem = [];
mpscore_rep = [];

for i in range(num_dem):
    mpmax = float("-inf");
    text = dem_tweets_v[i,:];
    notzero = np.nonzero(text)[0];
    for j in notzero:
        if (u_score.get(index_to_vocab[j]) is not None):
            if (mpmax < u_score[index_to_vocab[j]]):
                mpmax = u_score[index_to_vocab[j]]
    if(math.isinf(mpmax)):
        mpmax = float("inf");
    mpscore_dem.append(mpmax);

for i in range(num_rep):
    mpmax = float("-inf");
    text = rep_tweets_v[i,:];
    notzero = np.nonzero(text)[0];
    for j in notzero:
        if (u_score.get(index_to_vocab[j]) is not None):
            if (mpmax < u_score[index_to_vocab[j]]):
                mpmax = u_score[index_to_vocab[j]]
    if(math.isinf(mpmax)):
        mpmax = float("inf");
    mpscore_rep.append(mpmax);

In [16]:
all_mp_scores = np.array(mpscore_dem + mpscore_rep);
all_tweets = dem_tweets0 + rep_tweets0;
neutral_tweets = all_mp_scores.argsort()[:10];
for i in neutral_tweets:
    print(all_tweets[i])
    print(all_mp_scores[i])

#DumpTrump #UniteBlue #tcot 
0.0484133575203
#p1 #p2 #p21 #cspj #tcot # ...
0.0484133575203
#p1 #p2 #p21 #cspj #tcot ...
0.0484133575203
Hey!  #tcot 
0.0484133575203
#DumpTrump #tcot #UniteBlue #TNTweeters 
0.0484133575203
#UniteBlue #tcot #CircusAct #DumpTrump
0.0484133575203
#DumpTrump #UniteBlue #tcot #ClownCar 
0.0484133575203
#DumpTrump #NoMoreBushes #YouCruzYouLose #RubioRuse #UniteBlue #tcot
0.0484133575203
#DumpTrump #VoteBlue #RegisterToVote #UniteBlue #tcot
0.0484133575203
#dumptrump #lolgop #tcot 
0.0484133575203


In [18]:
for i in range(len(all_mp_scores)):
    if (math.isinf(all_mp_scores[i])):
        all_mp_scores[i] = float("-inf");
not_neutral_tweets = all_mp_scores.argsort()[::-1][:10]
for i in not_neutral_tweets:
    print(all_tweets[i])
    print(all_mp_scores[i])

To be the smartest president EVAR, Obama is really stupid.  What does that say about Thomas Jefferson?  #tcot #sgp #teaparty #hhrs
6.95742770418
So much angst about how we can better keep track of sex offenders - how about we don't let them out of jail? #tcot #hhrs
6.95742770418
 "Stand for freedom, and the American people will stand with you".  Mike Pence #tcot #teaparty #sgp #sot #hhrs
6.95742770418
Pelosi can't keep promise to "drain the swamp"   #gop #tcot #iamthemob #sgp #hcr #hhrs #glennbeck
6.95742770418
So President/CEO Obama fires Wagoner, sets employee pay, pulls GM out of NASCAR .. people still dont have a problem with this?? #tcot #hhrs
6.95742770418
really think we can "cut the rate of growth of national health care spending by 1.5 percentage points each year" I say no way #tcot #hhrs
6.95742770418
 Influential liberal sees the light on Obama  #news #tcot #gop #hhrs
6.95742770418
 read both speaches from today .. one used fear a LOT ... and it wasn't Cheney. That's the BHO